<a href="https://colab.research.google.com/github/jush-ua/CSST-101-3B-AY2526/blob/main/Assessment_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1**


In [4]:
def unify(p1, p2):
    f1, f2 = p1.split("(")[0], p2.split("(")[0]
    if f1 != f2: return "Cannot be unified"

    a1, a2 = p1[p1.find("(")+1:-1].split(","), p2[p2.find("(")+1:-1].split(",")
    if len(a1) != len(a2): return "Cannot be unified"

    subs = {}
    for x, y in zip(a1, a2):
        x, y = x.strip(), y.strip()
        if x == y: continue
        if x.islower(): subs[x] = y
        elif y.islower(): subs[y] = x
        else: return "Cannot be unified"

    return subs or "Already unified"

unification_result = unify("Eligible(x)", "Eligible(Ana)")

print(f"Unifying Eligible(x) and Eligible(Ana): {unification_result}")

if isinstance(unification_result, dict):
    consequence = "Scholarship(x)"
    derived_fact = consequence.replace('x', unification_result['x'])
    print(f"Inferred fact: {derived_fact}")

Unifying Eligible(x) and Eligible(Ana): {'x': 'Ana'}
Inferred fact: Scholarship(Ana)


# **Part 2**

In [10]:
rules = {
    "Eligible(X)": ["HighGrades(X)", "StudentLeader(X)", "Applied(X)"],
    "Scholarship(X)": ["Eligible(X)"],
    "FinancialSupport(X)": ["Scholarship(X)"]
}

facts = {
    "Juan": ["HighGrades(Juan)", "Applied(Juan)"],
    "Maria": ["HighGrades(Maria)", "DidNotApply(Maria)"],
    "Carlos": ["StudentLeader(Carlos)", "NoHighGrades(Carlos)"],
    "Ana": ["HighGrades(Ana)", "StudentLeader(Ana)", "Applied(Ana)"]
}

def has_fact(individual, fact_predicate):
    return fact_predicate in facts.get(individual, [])

def apply_rules(individual, current_facts):
    derived = []
    for head, body in rules.items():
        head_predicate = head.split("(")[0]
        if head_predicate in head.replace("X", individual):
            conditions_met = True
            for condition in body:
                condition_for_individual = condition.replace("X", individual)
                if not (condition_for_individual in current_facts or has_fact(individual, condition_for_individual)):
                    conditions_met = False
                    break

            if conditions_met:
                derived_fact = head.replace("X", individual)
                if derived_fact not in current_facts and not has_fact(individual, derived_fact):
                    derived.append(derived_fact)
    return derived

# --- Analysis for each individual ---

print("--- Analysis for Juan ---")
juan_derived = apply_rules("Juan", facts.get("Juan", []))
print(f"Initial facts for Juan: {facts.get('Juan', [])}")
print(f"Derived facts for Juan: {juan_derived}")
if "Eligible(Juan)" in juan_derived:
    print("Juan is Eligible.")
    juan_scholarship = apply_rules("Juan", facts.get("Juan", []) + juan_derived)
    print(f"Further derived facts for Juan: {juan_scholarship}")
else:
    print("Juan is not Eligible because he is not a Student Leader.")

print("\n--- Analysis for Ana ---")
ana_derived = apply_rules("Ana", facts.get("Ana", []))
print(f"Initial facts for Ana: {facts.get('Ana', [])}")
print(f"Derived facts for Ana: {ana_derived}")
if "Eligible(Ana)" in ana_derived:
    print("Ana is Eligible.")
    ana_scholarship = apply_rules("Ana", facts.get("Ana", []) + ana_derived)
    print(f"Further derived facts for Ana (after Eligible): {ana_scholarship}")
    ana_financial_support = apply_rules("Ana", facts.get("Ana", []) + ana_derived + ana_scholarship)
    print(f"Further derived facts for Ana (after Scholarship): {ana_financial_support}")

else:
     print("Ana is not Eligible (should not happen based on facts).")


print("\n--- Analysis for Maria ---")
print(f"Initial facts for Maria: {facts.get('Maria', [])}")
maria_derived = apply_rules("Maria", facts.get("Maria", []))
print(f"Derived facts for Maria: {maria_derived}")
print("Maria is not Eligible because she is not a Student Leader and hasn't Applied.")


print("\n--- Analysis for Carlos ---")
print(f"Initial facts for Carlos: {facts.get('Carlos', [])}")
carlos_derived = apply_rules("Carlos", facts.get("Carlos", []))
print(f"Derived facts for Carlos: {carlos_derived}")
print("Carlos is not Eligible because he does not have High Grades and hasn't Applied.")

--- Analysis for Juan ---
Initial facts for Juan: ['HighGrades(Juan)', 'Applied(Juan)']
Derived facts for Juan: []
Juan is not Eligible because he is not a Student Leader.

--- Analysis for Ana ---
Initial facts for Ana: ['HighGrades(Ana)', 'StudentLeader(Ana)', 'Applied(Ana)']
Derived facts for Ana: ['Eligible(Ana)']
Ana is Eligible.
Further derived facts for Ana (after Eligible): ['Scholarship(Ana)']
Further derived facts for Ana (after Scholarship): ['FinancialSupport(Ana)']

--- Analysis for Maria ---
Initial facts for Maria: ['HighGrades(Maria)', 'DidNotApply(Maria)']
Derived facts for Maria: []
Maria is not Eligible because she is not a Student Leader and hasn't Applied.

--- Analysis for Carlos ---
Initial facts for Carlos: ['StudentLeader(Carlos)', 'NoHighGrades(Carlos)']
Derived facts for Carlos: []
Carlos is not Eligible because he does not have High Grades and hasn't Applied.


# **Part 3**

In [13]:
rules = {
    "Eligible(X)": ["HighGrades(X)", "StudentLeader(X)", "Applied(X)"],
    "Scholarship(X)": ["Eligible(X)"],
    "FinancialSupport(X)": ["Scholarship(X)"]
}

facts = {
    "Juan": ["HighGrades(Juan)", "Applied(Juan)"],
    "Maria": ["HighGrades(Maria)", "DidNotApply(Maria)"],
    "Carlos": ["StudentLeader(Carlos)", "NoHighGrades(Carlos)"],
    "Ana": ["HighGrades(Ana)", "StudentLeader(Ana)", "Applied(Ana)"]
}

def backward_chaining(goal, facts, rules, level=0):
    indent = "  " * level
    print(f"{indent}Goal: {goal}")

    # 1. Check if the goal is in the initial facts
    goal_predicate = goal.split("(")[0]
    import re
    match = re.search(r'\((.*?)\)', goal)
    individual = match.group(1) if match else None

    if individual and goal in facts.get(individual, []):
        print(f"{indent}  -> Goal {goal} is a known fact.")
        return True

    # 2. Check if there is a rule whose head matches the goal
    for head, body in rules.items():
        head_predicate = head.split("(")[0]

        if head_predicate == goal_predicate:
            if individual and head.replace("X", individual) == goal:
                print(f"{indent}  -> Found rule: {head} :- {', '.join(body)}")
                all_subgoals_proven = True
                for subgoal in body:
                    substituted_subgoal = subgoal.replace("X", individual)
                    if not backward_chaining(substituted_subgoal, facts, rules, level + 1):
                        all_subgoals_proven = False
                        print(f"{indent}  -> Subgoal {substituted_subgoal} failed.")
                        break

                if all_subgoals_proven:
                    print(f"{indent}  -> All subgoals for rule {head} proven.")
                    return True

    # 3. If no rule or fact proves the goal
    print(f"{indent}  -> Goal {goal} could not be proven.")
    return False

print("--- Backward Chaining for Scholarship(Maria) ---")
backward_chaining("Scholarship(Maria)", facts, rules)

print("\n--- Backward Chaining for FinancialSupport(Ana) ---")
backward_chaining("FinancialSupport(Ana)", facts, rules)

--- Backward Chaining for Scholarship(Maria) ---
Goal: Scholarship(Maria)
  -> Found rule: Scholarship(X) :- Eligible(X)
  Goal: Eligible(Maria)
    -> Found rule: Eligible(X) :- HighGrades(X), StudentLeader(X), Applied(X)
    Goal: HighGrades(Maria)
      -> Goal HighGrades(Maria) is a known fact.
    Goal: StudentLeader(Maria)
      -> Goal StudentLeader(Maria) could not be proven.
    -> Subgoal StudentLeader(Maria) failed.
    -> Goal Eligible(Maria) could not be proven.
  -> Subgoal Eligible(Maria) failed.
  -> Goal Scholarship(Maria) could not be proven.

--- Backward Chaining for FinancialSupport(Ana) ---
Goal: FinancialSupport(Ana)
  -> Found rule: FinancialSupport(X) :- Scholarship(X)
  Goal: Scholarship(Ana)
    -> Found rule: Scholarship(X) :- Eligible(X)
    Goal: Eligible(Ana)
      -> Found rule: Eligible(X) :- HighGrades(X), StudentLeader(X), Applied(X)
      Goal: HighGrades(Ana)
        -> Goal HighGrades(Ana) is a known fact.
      Goal: StudentLeader(Ana)
        -> 

True

# **Part 4**

Here is the truth table for the rule: $(HighGrades \land Leader \land Applied) \rightarrow Eligible$

Let:
- H = HighGrades
- L = Leader (StudentLeader)
- A = Applied
- E = Eligible

The rule is $(H \land L \land A) \rightarrow E$.

| H     | L     | A     | $H \land L \land A$ | $(H \land L \land A) \rightarrow E$ (Assuming E is True when H, L, A are True, and can be True or False otherwise based on other rules/facts) |
|-------|-------|-------|---------------------|---------------------------------------------------------------------------------------------------------------------------------------|
| True  | True  | True  | True                | True (According to the rule, Eligible is true if H, L, and A are true)                                                                |
| True  | True  | False | False               | True                                                                                                                                    |
| True  | False | True  | False               | True                                                                                                                                    |
| True  | False | False | False               | True                                                                                                                                    |
| False | True  | True  | False               | True                                                                                                                                    |
| False | True  | False | False               | True                                                                                                                                    |
| False | False | True  | False               | True                                                                                                                                    |
| False | False | False | False               | True                                                                                                                                    |


In [17]:
facts = {
    "Juan": {"HighGrades": True, "StudentLeader": False, "Applied": True},
    "Maria": {"HighGrades": True, "StudentLeader": False, "Applied": False},
    "Carlos": {"HighGrades": False, "StudentLeader": True, "Applied": False},
    "Ana": {"HighGrades": True, "StudentLeader": True, "Applied": True}
}



print("--- Evaluating the rule (HighGrades AND StudentLeader AND Applied) -> Eligible for each student ---")

for student, student_facts in facts.items():
    high_grades = student_facts.get("HighGrades", False)
    student_leader = student_facts.get("StudentLeader", False)
    applied = student_facts.get("Applied", False)

    antecedent = high_grades and student_leader and applied
    print(f"\n--- {student} ---")
    print(f"Facts: HighGrades={high_grades}, StudentLeader={student_leader}, Applied={applied}")
    print(f"Antecedent (HighGrades AND StudentLeader AND Applied): {antecedent}")

    if antecedent:
        print(f"Conclusion: The antecedent is TRUE. According to the rule, {student} is Eligible.")
    else:
        print(f"Conclusion: The antecedent is FALSE. This rule does not conclude that {student} is Eligible.")

--- Evaluating the rule (HighGrades AND StudentLeader AND Applied) -> Eligible for each student ---

--- Juan ---
Facts: HighGrades=True, StudentLeader=False, Applied=True
Antecedent (HighGrades AND StudentLeader AND Applied): False
Conclusion: The antecedent is FALSE. This rule does not conclude that Juan is Eligible.

--- Maria ---
Facts: HighGrades=True, StudentLeader=False, Applied=False
Antecedent (HighGrades AND StudentLeader AND Applied): False
Conclusion: The antecedent is FALSE. This rule does not conclude that Maria is Eligible.

--- Carlos ---
Facts: HighGrades=False, StudentLeader=True, Applied=False
Antecedent (HighGrades AND StudentLeader AND Applied): False
Conclusion: The antecedent is FALSE. This rule does not conclude that Carlos is Eligible.

--- Ana ---
Facts: HighGrades=True, StudentLeader=True, Applied=True
Antecedent (HighGrades AND StudentLeader AND Applied): True
Conclusion: The antecedent is TRUE. According to the rule, Ana is Eligible.


# **Part 5**

Logical reasoning is important in AI because it allows machines to make decisions based on clear rules and evidence rather than guesswork. It helps AI systems solve problems, detect inconsistencies, and draw valid conclusions from given information. In real life, logical reasoning is equally important because it guides people in making fair, consistent, and well-structured decisions in situations like planning, problem-solving, or evaluating choices.